### Import python libraries to compare the modules:

In [138]:
import ncclient
from ncclient import manager
import xmltodict
import xml.dom.minidom
import pandas as pd 
import wget
import os
import difflib

In [139]:
m = manager.connect(
   host="10.95.86.212",
   port="830",
   username="tid",
   password="c22/pass",
   hostkey_verify=False)

In [141]:
for capability in m.server_capabilities:
        if "urn:ietf:params" in capability:
            #print capability
            pass
        else: 
            ns=capability.split("module=")[1].split("&revision=")[0]
            rev=capability.split("revision=")[1]
            filename="yang_cisco/"+ns+'@'+rev+".yang"
            if "openconfig" in capability:
                try:
                    schemay=m.get_schema(ns,rev)
                    schema_reply=xml.dom.minidom.parseString(schemay.xml)
                    file_out=schema_reply.getElementsByTagName("data")[0].childNodes[0].toxml()
                    file_out=file_out.replace("&quot;",'"')
                    with open(filename, 'w') as f:
                        #print "saving..."+ filename
                        f.write(file_out)
                except:
                    print "ERROR:"+filename

ERROR:yang_cisco/openconfig-rib-bgp@2016-04-11&deviations=cisco-xr-openconfig-rib-bgp-deviations.yang
ERROR:yang_cisco/openconfig-network-instance@2017-01-13&deviations=cisco-xr-openconfig-network-instance-deviations.yang
ERROR:yang_cisco/openconfig-platform@2018-06-29&deviations=cisco-xr-openconfig-platform-deviations.yang
ERROR:yang_cisco/openconfig-if-ethernet@2016-05-26&deviations=cisco-xr-openconfig-if-ethernet-deviations.yang
ERROR:yang_cisco/openconfig-platform-port@2018-01-20&deviations=cisco-xr-openconfig-platform-port-deviations.yang
ERROR:yang_cisco/openconfig-platform-transceiver@2018-11-25&deviations=cisco-xr-openconfig-platform-transceiver-deviations.yang
ERROR:yang_cisco/openconfig-telemetry@2016-02-04&deviations=cisco-xr-openconfig-telemetry-deviations.yang
ERROR:yang_cisco/openconfig-platform-psu@2018-01-16&deviations=cisco-xr-openconfig-platform-psu-deviations.yang
ERROR:yang_cisco/openconfig-rsvp-sr-ext@2017-03-06&deviations=cisco-xr-openconfig-rsvp-sr-ext-deviations

### Download the requiered files

In [83]:
def download_files(file_name, out):
    models = pd.read_excel(io=file_name, sheet_name="compare")
    for index, row in models.iterrows():
        moudule_name=row["SpecName"].split("@")[0]
        version=row["SpecName"].split("@")[1]
        out_file=out+moudule_name+'@'+version+".yang"
        url="https://www.yangcatalog.org/yang-search/module_details.php?module="+moudule_name+"@"+version
        r=requests.get(url)
        text = r.text
        plaintext = text.decode('utf8')
        links = re.findall("href=[\"\'](.*?)[\"\']", plaintext)
        links = list(dict.fromkeys(links))
        for link in links:
            if moudule_name+".yang" in link:
                url2=link
                r=requests.get(url2)
                try:
                    with open(out_file, 'w') as f:
                        print "saving..."+ out_file
                        f.write(r.text)
                except:
                    print "ERROR..."+ out_file
                    pass

In [117]:
file_name = "output.xlsx"
download_files(file_name,"yang_catalog/")

saving...yang_catalog/openconfig-bgp-types@2018-11-21.yang
saving...yang_catalog/openconfig-if-aggregate@2019-04-16.yang
saving...yang_catalog/openconfig-if-ethernet@2019-04-16.yang
ERROR...yang_catalog/openconfig-if-ethernet@2019-04-16.yang
saving...yang_catalog/openconfig-if-types@2018-11-21.yang
saving...yang_catalog/openconfig-inet-types@2019-04-25.yang
saving...yang_catalog/openconfig-interfaces@2019-11-19.yang
saving...yang_catalog/openconfig-lacp@2018-11-21.yang
saving...yang_catalog/openconfig-lldp@2018-11-21.yang
saving...yang_catalog/openconfig-lldp-types@2018-11-21.yang
saving...yang_catalog/openconfig-network-instance@2019-11-28.yang
saving...yang_catalog/openconfig-network-instance-l3@2018-11-21.yang
saving...yang_catalog/openconfig-network-instance-types@2018-11-21.yang
saving...yang_catalog/openconfig-packet-match@2018-11-21.yang
saving...yang_catalog/openconfig-platform@2019-04-16.yang
saving...yang_catalog/openconfig-platform-cpu@2018-11-21.yang
saving...yang_catalog/o

In [134]:
import filecmp

models = pd.read_excel(io=file_name, sheet_name="compare")        
for index, row in models.iterrows():
    dir1=row["SpecName"]
    dir2=row["CiscoImpl"]
    with open('yang_catalog/'+dir1+'.yang') as f, open('yang_cisco/'+dir2+'.yang') as g:
        flines = f.readlines()
        glines = g.readlines()

    d = difflib.HtmlDiff()
    a = d.make_table(flines, glines)
    with open ('diff/'+dir1+'.html','w') as h:
        h.write(a)

IOError: [Errno 2] No such file or directory: u'yang_cisco/cisco-xr-openconfig-if-ethernet-deviations@2016-05-26.yang'

In [137]:
!ls yang_cisco

cisco-xr-openconfig-acl-deviations@2018-02-07.yang
cisco-xr-openconfig-alarms-deviations@2019-11-14.yang
cisco-xr-openconfig-bfd-deviations@2019-05-06.yang
cisco-xr-openconfig-bgp-policy-deviations@2019-02-06.yang
cisco-xr-openconfig-if-ethernet-deviations@2016-05-16.yang
cisco-xr-openconfig-if-ip-deviations@2017-02-07.yang
cisco-xr-openconfig-if-ip-ext-deviations@2019-09-13.yang
cisco-xr-openconfig-interfaces-deviations@2016-05-16.yang
cisco-xr-openconfig-isis-policy-deviations@2019-06-10.yang
cisco-xr-openconfig-lacp-deviations@2019-09-10.yang
cisco-xr-openconfig-lldp-deviations@2017-03-08.yang
cisco-xr-openconfig-local-routing-deviations@2017-06-28.yang
cisco-xr-openconfig-network-instance-deviations@2017-10-16.yang
cisco-xr-openconfig-optical-amplifier-deviations@2019-04-05.yang
cisco-xr-openconfig-platform-cpu-deviations@2018-12-10.yang
cisco-xr-openconfig-platform-deviations@2018-12-05.yang
cisco-xr-openconfig-platform-port-deviations@2018-12-10.yang
cisco-xr-openconfig-platform-